In [14]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [15]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [16]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [17]:
def compute_item_weights(items_weights):
    items_weights = items_weights[items_weights['weight'] > 1]
    items_weights['weight'] = np.log(items_weights['weight'])
    
    items_weights['weight'] = items_weights['weight'] / items_weights['weight'].sum()
    
    return items_weights

def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    recs = np.random.choice(items_weights['item_id'], size=n, replace=True, p=items_weights['weight'].tolist())
    
    return recs.tolist()

In [18]:
%%time

items_weights = data_train.groupby('item_id')['sales_value'].sum().reset_index()
items_weights.columns = ['item_id', 'weight']

#print(items_weights)

items_weights = compute_item_weights(items_weights)

#print(items_weights)
#print(items_weights['weight'].describe())

Wall time: 163 ms


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [19]:
import re

result = pd.read_csv('predictions/predictions_basic.csv')

# преобразуем строки в списке
def parse_row(row):
    if row == '[]':
        return list()
    
    return list(map(int, re.split(r'[\s|,]+', row.strip('[ ]'))))
    
for col in result.columns[1:]:
    result[col] = result[col].apply(lambda x: parse_row(x))
    
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[843222, 15596951, 12670127, 1125478, 1103416]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1127831, 995242]","[1082185, 6666, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 6666, 1098066]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1079382, 824555, 1121865, 13038785, 884118]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 6666, 826249]","[1082185, 981760, 1098066, 826249, 6666]"


In [20]:
# добавим столбец "weighted_random_recommendation"
result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5))
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[843222, 15596951, 12670127, 1125478, 1103416]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1127831, 995242]","[1082185, 6666, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 6666, 1098066]","[6979162, 15971851, 12988325, 855904, 987061]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1079382, 824555, 1121865, 13038785, 884118]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 6666, 826249]","[1082185, 981760, 1098066, 826249, 6666]","[13007721, 1000736, 913052, 1007360, 7167835]"


In [47]:
from mm_metrics import precision_at_k, recall_at_k, ap_k, ndcg_at_k, reciprocal_rank

res_columns = ['precision_at_k', 'recall_at_k', 'ap_k', 'ndcg_at_k', 'reciprocal_rank']
res_df = pd.DataFrame(index=result.columns[1:], columns=res_columns)
#res_df

for metric_col in res_df.columns:
    for algoritm_col in result.columns[1:]:
        # вызов функции через eval :-)
        res_df[metric_col][algoritm_col] = round(result.apply(lambda row: eval(f'{metric_col}(row["{algoritm_col}"], row["actual"], k=5)'), axis=1).mean(),4)

#actual = get_algo_metric_value(result, 'actual')
print("k=5")
res_df

k=5


,precision_at_k,recall_at_k,ap_k,ndcg_at_k,reciprocal_rank
actual,1.0,0.271,1.0,0.627,1.0
random_recommendation,0.0008,0.0001,0.0022,0.0024,0.0022
popular_recommendation,0.1552,0.025,0.2568,0.4495,0.286
itemitem,0.1368,0.0157,0.221,0.483,0.2136
cosine,0.1329,0.0148,0.3743,0.2597,0.3955
tfidf,0.139,0.0154,0.3816,0.2751,0.4013
weighted_random_recommendation,0.0018,0.0001,0.0029,0.0083,0.0029


**TFIDFRecommender Выдает лучшие результаты**